In [1]:
# Colab setup

!pip install gym > /dev/null 2>&1

!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

!git clone https://github.com/yfletberliac/rlss2019-hands-on.git > /dev/null 2>&1
!pip install -q torch==1.1.0 torchvision pyvirtualdisplay piglet > /dev/null 2>&1

# Deep Q-Learning (DQN)


In DQN, the $Q$-function is parameterized by a neural network of parameters $\theta$. The network takes as input a state $s$ and outputs $Q(s, a, \theta)$ for all actions $a$. 

The network is trained in way that is similar to Fitted Q Iteration. At each time $T$, the agent has observed the transitions $(s_t, a_t, r_t, s_t')_{t=1}^T$, which are stored in a __replay buffer__.

In addition to the network with parameters $\theta$, DQN keeps another network with the same architecture and parameter $\tilde{\theta}$, called __target network__. 
To update the parameters $\theta$, we sample $N$ transitions from the __replay buffer__, we define the loss 

$$
L(\theta) = \sum_{i=1}^N [Q(s_i, a_i, \theta) - (r_i + \gamma\max_{a'}Q(s'_i,a', \tilde{\theta}))]^2
$$

and update 

$$
\theta \gets \theta + \eta \nabla L(\theta).
$$


Every $C$ iterations, the target network is updated as $\tilde{\theta} \gets \theta$. 

At each time $t$, DQN updates the networks as described above, selects an action according to an $\epsilon$-greedy, plays the action and stores the new data in the replay buffer.

In [2]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import gym
import random
from copy import deepcopy

from gym.wrappers import Monitor
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from IPython.display import clear_output
from pathlib import Path
import base64

## Step 1: Define the parameters

In [3]:
# Environment
env = gym.make("CartPole-v0")

# Discount factor
GAMMA = 0.99

# Batch size
BATCH_SIZE = 256
# Capacity of the replay buffer
BUFFER_CAPACITY = 10000
# Update target net every ... episodes
UPDATE_TARGET_EVERY = 20

# Initial value of epsilon
EPSILON_START = 1.0
# Parameter to decrease epsilon
DECREASE_EPSILON = 200
# Minimum value of epislon
EPSILON_MIN = 0.05

# Number of training episodes
N_EPISODES = 200

# Learning rate
LEARNING_RATE = 0.1

/home/omardrwch/miniconda3/envs/rlpython/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Step 2: Define the replay buffer

In [4]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, state, action, reward, next_state):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = (state, action, reward, next_state)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
# create instance of replay buffer
replay_buffer = ReplayBuffer(BUFFER_CAPACITY)

## Step 3: Define the neural network architecture, objective and optimizer

In [6]:
class Net(nn.Module):
    """
    Basic neural net.
    """
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [7]:
# create network and target network
hidden_size = 128
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

q_net = Net(obs_size, hidden_size, n_actions)
target_net = Net(obs_size, hidden_size, n_actions)

# objective and optimizer
objective = nn.MSELoss()
optimizer = optim.Adam(params=q_net.parameters(), lr=LEARNING_RATE)

## Step 4: Implement DQN

In [8]:
#
#  Some useful functions
#

def get_q(states):
    """
    Compute Q function for a list of states
    """
    with torch.no_grad():
        states_v = torch.FloatTensor([states])
        output = q_net.forward(states_v).data.numpy()  # shape (1, len(states), dim_state)
    return output[0, :, :]  # shape (len(states), dim_state)

def dqn_policy(s):
    """
    Choose action at s using q_net
    """
    if not isinstance(s, np.ndarray):
        raise TypeError("Vector encoding expected for state variable.")
    q = get_q([s])[0]
    return q.argmax()

def eval_dqn(n_sim=5):
    """
    Monte Carlo evaluation of DQN agent
    """
    rewards = []
    copy_env = deepcopy(env)
    for sim in range(n_sim):
        done = False
        state = copy_env.reset()
        sim_rewards = 0.0
        while not done:
            action = dqn_policy(state)
            next_state, reward, done, _ = copy_env.step(action)
            sim_rewards += reward
            state = next_state
        rewards.append(sim_rewards)
    return rewards

In [9]:
def choose_action(state, epsilon):
    """
    TO BE IMPLEMENTED
    
    Return action according to an epsilon-greedy exploration policy
    """
    return 0
    

def update(state, action, reward, next_state, done):
    """
    TO BE COMPLETED
    """
    
    # add data to replay buffer
    if done:
        next_state = None
    replay_buffer.push(state, action, reward, next_state)
    
    if len(replay_buffer) < BATCH_SIZE:
        return np.inf
    
    # get batch
    transitions = replay_buffer.sample(BATCH_SIZE)
    
    # Compute loss - TO BE IMPLEMENTED!
    values  = torch.zeros(BATCH_SIZE)   # to be computed using batch
    targets = torch.zeros(BATCH_SIZE)   # to be computed using batch
    loss = objective(values, targets)
     
    # Optimize the model - UNCOMMENT!
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
    return loss.data.numpy()

In [10]:
#
# Train
# 

EVAL_EVERY = 5
REWARD_THRESHOLD = 199

def train():
    state = env.reset()
    epsilon = EPSILON_START
    ep = 0
    total_time = 0
    while ep < N_EPISODES:
        action = choose_action(state, epsilon)

        # take action and update replay buffer and networks
        next_state, reward, done, _ = env.step(action)
        loss = update(state, action, reward, next_state, done)

        # update state
        state = next_state

        # end episode if done
        if done:
            state = env.reset()
            ep   += 1
            rewards = eval_dqn()
            if ( (ep+1)% EVAL_EVERY == 0):
                print("episode =", ep+1, ", reward = ", np.mean(rewards))
                if np.mean(rewards) >= REWARD_THRESHOLD:
                    break

            # update target network
            if ep % UPDATE_TARGET_EVERY == 0:
                target_net.load_state_dict(q_net.state_dict())
            # decrease epsilon
            epsilon = EPSILON_MIN + (EPSILON_START - EPSILON_MIN) * \
                            np.exp(-1. * ep / DECREASE_EPSILON )    

        total_time += 1

train()
rewards = eval_dqn(20)
print("")
print("mean reward after training = ", np.mean(rewards))

episode = 5 , reward =  9.4
episode = 10 , reward =  9.0
episode = 15 , reward =  9.8
episode = 20 , reward =  9.2
episode = 25 , reward =  9.0
episode = 30 , reward =  10.0
episode = 35 , reward =  8.8
episode = 40 , reward =  8.8
episode = 45 , reward =  9.2
episode = 50 , reward =  9.4
episode = 55 , reward =  9.2
episode = 60 , reward =  8.6
episode = 65 , reward =  9.2
episode = 70 , reward =  9.2
episode = 75 , reward =  8.8
episode = 80 , reward =  9.0
episode = 85 , reward =  9.6
episode = 90 , reward =  9.4
episode = 95 , reward =  9.4
episode = 100 , reward =  9.4
episode = 105 , reward =  9.2
episode = 110 , reward =  9.8
episode = 115 , reward =  9.4
episode = 120 , reward =  9.0
episode = 125 , reward =  9.2
episode = 130 , reward =  9.4
episode = 135 , reward =  9.8
episode = 140 , reward =  9.2
episode = 145 , reward =  9.4
episode = 150 , reward =  9.6
episode = 155 , reward =  9.4
episode = 160 , reward =  9.6
episode = 165 , reward =  9.8
episode = 170 , reward =  10.

## Visualizing the agent

In [11]:
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    html = []
    for mp4 in Path("videos").glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))
    
env = Monitor(env, './videos', force=True, video_callable=lambda episode: True)

for episode in range(1):
    done = False
    state = env.reset()
    while not done:
        action = dqn_policy(state)
        state, reward, done, info = env.step(action)
env.close()
show_video()